#### Note: Latest May 21 (7.30pm) - OneHot Encoding (not...still get_dummies). Creating Kelly's 3 breed categories



## Reading in data and MAJOR CLEANING


In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import datetime as dt

In [3]:
# Read in csv
df = pd.read_csv("Resources/Animal_Shelter_Louisville.csv")
df.head(10)


,AnimalID,AnimalType,IntakeDate,IntakeType,IntakeSubtype,PrimaryColor,PrimaryBreed,SecondaryBreed,Gender,SecondaryColor,...,IntakeInternalStatus,IntakeAsilomarStatus,ReproductiveStatusAtIntake,OutcomeDate,OutcomeType,OutcomeSubtype,OutcomeReason,OutcomeInternalStatus,OutcomeAsilomarStatus,ReproductiveStatusAtOutcome
0,A366370,CAT,7/11/2008 10:50,STRAY,OTC,WHITE,DOMESTIC SHORTHAIR,NaN,NEUTERED MALE,BROWN,...,FEARFUL,HEALTHY,ALTERED,12/11/2008 15:46,EUTH,FERAL,NaN,NaN,UNHEALTHY/UNTREATABLE,ALTERED
1,A366531,CAT,10/11/2008 10:20,STRAY,OTC,BLACK,DOMESTIC SHORTHAIR,DOMESTIC SHORTHAIR,UNKNOWN,NaN,...,NORMAL,HEALTHY,UNKNOWN,19/11/2008 20:10,EUTH,CONTAG DIS,NaN,SICK,HEALTHY,UNKNOWN
2,A532367,BIRD,23/7/2014 23:21,CONFISCATE,CRUELTY,RED,CHICKEN,NaN,MALE,BLACK,...,OTHER,HEALTHY,FERTILE,5/11/2014 15:49,TRANSFER,NaN,NaN,NaN,HEALTHY,FERTILE
3,A532474,OTHER,24/7/2014 18:29,ET REQUEST,NaN,BROWN,BAT,NaN,UNKNOWN,NaN,...,OTHER,HEALTHY,UNKNOWN,24/7/2014 23:59,EUTH,MEDICAL,NaN,OTHER,HEALTHY,UNKNOWN
4,A281756,DOG,11/9/2006 18:10,OWNER SUR,OTC,WHITE,PIT BULL TERRIER,NaN,MALE,BROWN,...,NORMAL,HEALTHY,FERTILE,12/9/2006 13:44,EUTH,TIME/SPACE,NaN,NaN,HEALTHY,FERTILE
5,A451184,BIRD,29/1/2012 15:25,OWNER SUR,FIELD,BLACK,CHICKEN,NaN,UNKNOWN,WHITE,...,NORMAL,HEALTHY,UNKNOWN,22/2/2012 23:59,TRANSFER,RESCUE GRP,NaN,NORMAL,HEALTHY,UNKNOWN
6,A256128,DOG,26/11/2005 12:35,STRAY,FIELD,BROWN,AMERICAN PIT BULL TERRIER,MIX,MALE,WHITE,...,NORMAL,HEALTHY,FERTILE,8/12/2005 23:59,EUTH,MEDICAL,NaN,NaN,HEALTHY,FERTILE
7,A314432,CAT,15/6/2007 17:13,OWNER SUR,OTC,BROWN TABBY,DOMESTIC SHORTHAIR,NaN,UNKNOWN,NaN,...,NORMAL,HEALTHY,UNKNOWN,15/6/2007 18:07,EUTH,TIME/SPACE,NaN,NaN,HEALTHY,UNKNOWN
8,A316619,DOG,29/6/2007 20:10,STRAY,FIELD,WHITE,LABRADOR RETRIEVER,MIX,MALE,TAN,...,FEARFUL,HEALTHY,FERTILE,4/7/2007 13:12,EUTH,TIME/SPACE,NaN,NaN,HEALTHY,FERTILE
9,A317335,CAT,5/7/2007 21:30,STRAY,OTC,GRAY,DOMESTIC SHORTHAIR,NaN,UNKNOWN,NaN,...,NORMAL,HEALTHY,UNKNOWN,11/7/2007 9:19,EUTH,TIME/SPACE,NaN,NaN,HEALTHY,UNKNOWN


In [ ]:
df.isna().sum()

In [ ]:
df.columns

In [ ]:
# Drop unnecessary columns
df2 = df.drop(columns=['AnimalID',
                       'IntakeType', 
                       'IntakeSubtype', 
                       'OutcomeSubtype',
                      'SecondaryColor',
                       'IntakeReason',
                      'ReproductiveStatusAtOutcome',
                       'OutcomeReason',
                      'OutcomeInternalStatus',
                      'OutcomeAsilomarStatus'])

In [ ]:
df2.head()


In [ ]:
# Change object to datetime so we can calculate AgeInMonth and DurationInShelter
df2["IntakeDate"] = pd.to_datetime(df2["IntakeDate"])
df2["DOB"] = pd.to_datetime(df2["DOB"])
df2["OutcomeDate"] = pd.to_datetime(df2["OutcomeDate"])

In [ ]:
df2.dtypes

In [ ]:
df2.head()

In [ ]:
df2.groupby("AnimalType").count()

In [ ]:
# Drop all rows NOT dog!
df3 = df2.drop(df2[(df2['AnimalType'] == "BIRD") | 
                   (df2['AnimalType'] == "CAT") |
                   (df2['AnimalType'] == "FERRET") |
                   (df2['AnimalType'] == "LIVESTOCK") |
                   (df2['AnimalType'] == "OTHER") |
                   (df2['AnimalType'] == "RABBIT") |
                   (df2['AnimalType'] == "REPTILE") |
                   (df2['AnimalType'] == "RODENT")].index)

In [ ]:
df3.groupby("AnimalType").count()

## Cleaning WITHIN dog
1. Drop unnessary columns
2. Drop rows where "Outcome_Type" is null
3. Drop rows where "Outcome_Type" is Return_To_Owner

In [ ]:
df3 = df3.drop(columns=['AnimalType'])

In [ ]:
# Quick check for missing values
df3.isna().sum()

In [ ]:
df3.columns

In [ ]:
# Drop rows where Outcome_Type is null

df3 = df3.loc[df3.OutcomeType.notna(), ["IntakeDate", 
                                        'PrimaryColor',
                                        "PrimaryBreed", 
                                        "SecondaryBreed", 
                                        "Gender",
                                        'DOB',
                                        "IntakeInternalStatus", 
                                        'IntakeAsilomarStatus', 
                                        'ReproductiveStatusAtIntake', 
                                        'OutcomeDate',
                                        'OutcomeType']]


In [ ]:
df3.shape

In [ ]:
# Check for more nulls

# df3.apply(lambda x: sum(x.isnull()/len(df3)))
df3.isna().sum()

In [ ]:
# # df2.groupby('Outcome_Type').count()
# df2.groupby('Color').count().sort_values('Outcome_Type', ascending=False)
df3.groupby('OutcomeType').count()

In [ ]:
# Drop all rows 'RTO'. Assuming that chip is checked at intake

df3 = df3.drop(df3[df3.OutcomeType == "RTO"].index)

In [ ]:
df3.shape

In [ ]:
# More nulls?  Keeping secondary breed for now
# df3.apply(lambda x: sum(x.isnull()/len(df3)))
df3.isna().sum()

In [ ]:
# Drop NA values in DOB (can't have nulls to calculate AgeInMonths)
df3 = df3.dropna(axis=0, subset=["DOB", 'PrimaryColor', 'OutcomeDate'])
df3.shape


In [ ]:
df3.isna().sum()

In [ ]:
# Create AgeInMonth column since data is cleaner now
df3['AgeInMonths'] = round(((df3['IntakeDate'] - df3['DOB']).dt.days)/30, 2)
df3.head()

In [ ]:
# Create DurationInShelter column
df3['DurationInShelter'] = round(((df3['OutcomeDate'] - df3['IntakeDate']).dt.days), 2)
df3.head()

In [ ]:
df4 = df3.drop(columns=['IntakeDate',
                       'DOB', 
                       'OutcomeDate'])

In [ ]:
# More nulls?  Keeping secondary breed for now
df4.isna().sum()

In [ ]:
df4.shape

## Add new features? Feature Engineering
1. New feature? - Mixed/Pure Breed? - COMPLETED!! 
2. Split breeds - COMPLETED!!
3. Color -  We have 333 unique color combinations. Hot encoding "Color" will create additional X features if we didn't do anything.  This would be unlikely to create a good performing model.
    *  Option 1 - We take any values occurring fewer than 300 times and places them into the “other” label. We use this to create a new feature called “Top_colors”.  Doing this would reduce 333 unique colors to 30 unique values - COMPLETED!
    *  Option 2 - Split to primary color vs secondary color? - COMPLETED!
4. Creating age groups - COMPLETED!!
5. Breeds - Hot encoding "Breed" will create additional 1,380 new features if we didn't do anything.  
6. New feature? - Single color/Multi color?

#### Creating a Mix/Pure Breed column - As of May 20 9pm, MIX doesn't do anything cause assuming all breeds are mixed.
#### Creating Kelly's 3 breed categories: 0 = mix, secondary not listed ; 1 = secondary breed listed; 2 = purebred

1. Create new MIX column (Kelly's 3 categories)
2. Split breed into breed_primary and breed_second


#### solution - Creating a Breed_category column AND Split into Breed_primary/Breed_second
* 0 = Breed has the word "Mix"
* 1 = If there are 2 breeds listed
* 2 = Only 1 breed in the primary (i.e. None in secondary OR Pure)

In [ ]:
df4.loc[500:1000, :]

In [ ]:
df4.isna().sum()

In [ ]:
# df3.SecondaryBreed.value_counts().sort_values(ascending=False).head(20)
# df3.SecondaryBreed.count()

In [ ]:
# Add new column called BreedCategory, set all to Mix

df4["BreedCategory"]= "Two"
df4.loc[500:1000, :]

In [ ]:
# Find rows containing word "Nan" in SecondaryBreed and set to Pure

df4.loc[df4['SecondaryBreed'].isnull(),'BreedCategory'] = "Pure"
df4.loc[500:1000, :]



In [ ]:
# Find rows containing word "Mix"
df4.loc[df4["SecondaryBreed"].str.contains("MIX",  na=False), "BreedCategory"] = "Mix"
df4.loc[500:1000, :]

In [ ]:
# Drop SeondaryBreed column since Breed Category is available
df4 = df4.drop(columns=['SecondaryBreed'])
df4.shape

In [ ]:
df4.head()

In [ ]:
df4.isna().sum()

### Trying to Group "IntakeInternalStatus"

In [ ]:
df4.groupby('IntakeInternalStatus').count()

In [ ]:
# df5.groupby("IntakeInternalStatus").count()
# df5.IntakeInternalStatus.value_counts().to_dict()

df4['IntakeStatus'] = "Sick"

In [ ]:
# Group all AGG together
df4.loc[df4["IntakeInternalStatus"].str.contains("AGG"), "IntakeStatus"] = "Aggressive"

In [ ]:
# Group all Normal together
df4.loc[df4["IntakeInternalStatus"].str.contains("NORMAL"), "IntakeStatus"] = "Normal"

In [ ]:
df4.loc[df4["IntakeInternalStatus"] == "NORMAL"]

In [ ]:
# Add these to Aggresive
df4.loc[df4["IntakeInternalStatus"].str.contains("TERITORIAL"), "IntakeStatus"] = "Aggressive"
df4.loc[df4["IntakeInternalStatus"].str.contains("FERAL"), "IntakeStatus"] = "Aggressive"
df4.loc[df4["IntakeInternalStatus"].str.contains("FEARFUL"), "IntakeStatus"] = "Aggressive"

# Group these to Pregnant
df4.loc[df4["IntakeInternalStatus"].str.contains("NURSING"), "IntakeStatus"] = "Pregnant"
df4.loc[df4["IntakeInternalStatus"].str.contains("PREGNANT"), "IntakeStatus"] = "Pregnant"

# Group these to Other
df4.loc[df4["IntakeInternalStatus"].str.contains("AGED"), "IntakeStatus"] = "Other"
df4.loc[df4["IntakeInternalStatus"].str.contains("DEAD"), "IntakeStatus"] = "Other"
df4.loc[df4["IntakeInternalStatus"].str.contains("OTHER"), "IntakeStatus"] = "Other"
df4.loc[df4["IntakeInternalStatus"].str.contains("OBESE"), "IntakeStatus"] = "Other"

 




In [ ]:
df4.groupby('IntakeStatus').count()

In [ ]:
df4.head()

#### Handling Color - Option 1 (Grouping colors)
* How does option1 effect RF/LogReg models?


In [ ]:
# # How many unique color combinations?  - 333
# color_counts = df4['Color'].value_counts()
# # len(color_counts)
# print(df4['Color'].nunique())
# color_counts.head(20)

In [ ]:
# # # Handling "Color" Option 1
# color_others = set(color_counts[color_counts < 1200].index)
# df4['Top_colors'] = df4['Color'].replace(list(color_others), 'Others')
# print(df4['Top_colors'].nunique())

In [ ]:
# # Add new column called Color_new and set to Color
# df5 = df4.copy()
# df5["Color_new"]=df4["Color"]
# df5.head()
# df5.loc[320:330, :]

In [ ]:
# Can't do this code here.  It takes COMBI tan/white and changes that to tan!  WRONG!!  I only want tan column


# Find rows containing colors: Gold/Yellow/Tan/Fawn/Buff/Apricot/Cream and set all to YELLOW
# df5.loc[df5["Color"].str.contains('Gold|Yellow|Tan|Fawn|Buff|Apricot|Cream'), "Color_new"] = "Tan"
# df5.loc[264:267, :]

#### Creating age categories
* Baby: <=6months
* Young: 7months-24 months
* Adult: 2 years-7years
* Senior: 7+

In [ ]:
df4['Pet_age_category'] = pd.cut(x=df4['AgeInMonths'], bins=[0, 6, 24, 84,240], labels=['Baby', 'Young', 'Adult', 'Senior'])

In [ ]:
df4.head(10)


In [ ]:
# Drop AgeInMonths column since you have Pet_age_category


## Changing to Binary Outcome - Take the dog or Don't take the dog?
* Positive Outcome is Adoption - Take in the dog
* Negative Outcome is a combination of Transfer, Euthanized and Death categories - Don't take in the dog

In [ ]:
# Create new BINARY_OUTCOME column and set to 0

df4["BinaryOutcome"]="Deny"
df4.head()

In [ ]:
# Find rows containing word "Adoption" and set to 1 (for Positive Outcome)
# All other rows will be 0 (for Negative Outcome)

df4.loc[df4["OutcomeType"].str.contains("ADOPTION"), "BinaryOutcome"] = "Take"

In [ ]:
df4.head()

In [ ]:
# Split gender because ReproductiveStatusAtIntake is similar to Gender
df4["Sex"] = "Male"

In [ ]:
df4.head()

In [ ]:
# Find rows containing word "FEMALE" and set to Female
df4.loc[df4["Gender"].str.contains("FEMALE"), "Sex"] = "Female"

In [ ]:
df4.head()

In [ ]:
# df3.to_csv('LouisvilleClean.csv')

In [ ]:
df4.columns

In [ ]:
# Final dropping of duplicated columns
df5 = df4.drop(columns=[ 'Gender', 'IntakeInternalStatus','IntakeAsilomarStatus','ReproductiveStatusAtIntake','OutcomeType','AgeInMonths','DurationInShelter'])

## Still get_dummies

In [ ]:
df5.head(15)

In [ ]:
df6 = pd.get_dummies(df5[['PrimaryColor',
                          'PrimaryBreed', 
                          'IntakeStatus', 
                          'BreedCategory',
                          'Pet_age_category',
                          'BinaryOutcome',
                          "Sex"]], drop_first=True)

In [ ]:
df6.columns


In [ ]:
# Define X and y
X = df6.drop(['BinaryOutcome_Take'], axis="columns")
y = df6[['BinaryOutcome_Take']]

In [ ]:
X.shape

In [ ]:
y.shape

## Random Forest model


In [ ]:
# Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
# Import, initialize, fit and predict
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators = 100, random_state = 42) 
rf_model.fit(X_train, y_train)

In [ ]:
predict_y_test = rf_model.predict(X_test)

In [ ]:
# Validate - run accuracy score
from sklearn import metrics
print("Accuracy score: ", metrics.accuracy_score(y_test, predict_y_test))

In [ ]:
from rfpimp import *

In [ ]:
# total number of nodes in all decision trees of the forest and the height (in nodes) of the typical tree.
# The tree height matters because that is the path taken by the RF prediction mechanism and so tree height effects prediction speed.
print(f"{rfnnodes(rf_model):,d} tree nodes and {np.median(rfmaxdepths(rf_model))} median tree height")

In [ ]:
# RF Feature importance?
feature_list = list(X.columns)
feature_importance = pd.Series(rf_model.feature_importances_, index=feature_list).sort_values(ascending=False) 
print(feature_importance[0:10])

In [ ]:
# Validate with cross validation

from sklearn.model_selection import cross_val_score

scores = cross_val_score(rf_model, X, y, cv=10, scoring="accuracy")
print(scores)

In [ ]:
scores.mean()

## SVM


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42)  

target = df6["BinaryOutcome_Take"]



In [ ]:
from sklearn.svm import SVC 
svm_model = SVC(kernel='rbf')
svm_model.fit(X_train, y_train)

In [ ]:
# SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
#   decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
#   max_iter=-1, probability=False, random_state=None, shrinking=True,
#   tol=0.001, verbose=False)

In [ ]:
# Model Accuracy
print('Test Acc: %.3f' % svm_model.score(X_test, y_test))

In [ ]:
 # Calculate classification report
from sklearn.metrics import classification_report
predictions = svm_model.predict(X_test)
print(classification_report(y_test, predictions))